In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType, StringType, BooleanType, TimestampType, IntegerType, ShortType
from delta.tables import DeltaTable
from datetime import datetime
from pyspark.sql.functions import col, regexp_replace, sha2, when, lit, uuid

CATALOGO_ORIGEM = "v_credit"
SCHEMA_ORIGEM = "bronze"
TABELA_ORIGEM = "chamados"

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_chamado"
TABELA_INVALIDOS_DESTINO = "tb_chamado_invalidos"

nome_tabela_origem = f"{CATALOGO_ORIGEM}.{SCHEMA_ORIGEM}.{TABELA_ORIGEM}"
nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

timestamp_atual = F.current_timestamp()


In [0]:
# DataFrame inicial (como definido no seu código)
df_chamado = spark.table("v_credit.bronze.chamados")

chamado_limpo = (
    df_chamado
    # 1. Renomeação de Colunas
    .withColumnRenamed("id_chamado", "cd_chamado")
    .withColumnRenamed("id_cliente", "cd_cliente")
    .withColumnRenamed("motivo", "ds_motivo")
    .withColumnRenamed("canal", "cd_canal") # Novo
    .withColumnRenamed("resolvido", "st_resolvido")
    .withColumnRenamed("tempo_espera", "tm_espera") # Novo, alinhado ao metadado
    .withColumnRenamed("tempo_atendimento", "tm_duracao") # Novo (renomeia o antigo para o novo tm_duracao)
    .withColumnRenamed("id_atendente", "cd_atendente")
    .withColumnRenamed("ingestion_timestamp", "dt_ingestion")
    .withColumnRenamed("origem", "dc_origem")

    .drop("hora_abertura_chamado")
    .drop("hora_inicio_atendimento")
    .drop("hora_finalizacao_atendimento")
    .drop("ctid_fivetran_id")
    .drop("_fivetran_deleted")
    .drop("_fivetran_synced")
    
    .withColumn("cd_cliente", F.col("cd_cliente").cast(StringType()))
    .withColumn("ds_motivo", F.col("ds_motivo").cast(StringType()))
    
    .withColumn("dt_ingestion", F.col("dt_ingestion").cast(TimestampType()))
    .withColumn("dc_origem", F.col("dc_origem").cast(StringType()))
)

chamado_limpo.printSchema()

In [0]:
chamado_limpo = chamado_limpo.withColumn(
    "st_resolvido",
    regexp_replace(F.col("st_resolvido"), "(?i)n.o", "false")
).withColumn(
    "st_resolvido",
    regexp_replace(F.col("st_resolvido"), "(?i)sim", "true")
).withColumn(
    "st_resolvido",
    F.col("st_resolvido").cast(BooleanType())
)

chamado_limpo = chamado_limpo.withColumn("cd_cliente",  F.sha2(F.col("cd_cliente").cast(StringType()), 256))

In [0]:
df_log = spark.table("v_credit.silver.tb_chamado_log")

# 2. Realizar o Join
# Fazemos um left join para trazer as datas do log para o chamado_limpo
# Selecionamos apenas as colunas necessárias do log para evitar duplicidade ou ambiguidade
chamado_limpo = chamado_limpo.join(
    df_log.select("cd_chamado", "dh_abertura", "dh_inicio", "dh_fim"),
    on="cd_chamado",
    how="left"
)

# 3. Calcular tm_espera e tm_duracao e converter para bigint
chamado_limpo = (
    chamado_limpo
    # tm_espera: Diferença entre início do atendimento e abertura (em segundos)
    .withColumn("tm_espera", (F.col("dh_inicio").cast("long") - F.col("dh_abertura").cast("long")))
    
    # tm_duracao: Diferença entre fim e início do atendimento (em segundos)
    .withColumn("tm_duracao", (F.col("dh_fim").cast("long") - F.col("dh_inicio").cast("long")))
    
    # Converter explicitamente para bigint (LongType)
    .withColumn("tm_espera", F.col("tm_espera").cast(LongType()))
    .withColumn("tm_duracao", F.col("tm_duracao").cast(LongType()))
)

display(chamado_limpo)

In [0]:
from pyspark.sql import functions as F

# 1. Tratamento de caracteres inválidos (encoding) na coluna ds_motivo_rel
# Normaliza as strings antes do join para aumentar as chances de correspondência
chamado_limpo = (
    chamado_limpo
    .withColumn("ds_motivo", F.regexp_replace(F.col("ds_motivo"), "Contrata..o", "Contratação"))
    .withColumn("ds_motivo", F.regexp_replace(F.col("ds_motivo"), "Altera..o", "Alteração"))
    .withColumn("ds_motivo", F.regexp_replace(F.col("ds_motivo"), "Contesta..o", "Contestação"))
    .withColumn("ds_motivo", F.regexp_replace(F.col("ds_motivo"), "(?i)cart.o", "cartão")) # (?i) para case insensitive se necessário
    .withColumn("ds_motivo", F.regexp_replace(F.col("ds_motivo"), "(?i)n.o", "não"))
    .withColumn("ds_motivo", F.regexp_replace(F.col("ds_motivo"), "D.vidas", "Dúvidas"))
)

# 2. Carregar a tabela Silver de Motivos
tb_motivo = spark.table("v_credit.silver.tb_motivo")

# 3. Join corrigido usando 'ds_motivo'
# Compara a descrição do chamado (ds_motivo_rel) com a descrição da tabela de motivos (ds_motivo)
chamado_limpo = (
    chamado_limpo.alias("c")
    .join(
        tb_motivo.alias("m"),
        # Normalização (lower + trim) para garantir o match
        F.lower(F.trim(F.col("c.ds_motivo"))) == F.lower(F.trim(F.col("m.ds_motivo"))),
        "left"
    )
    .select(
        # Seleciona todas as colunas originais do chamado, exceto a descrição antiga
        *[F.col(f"c.{col}") for col in chamado_limpo.columns if col != "ds_motivo"],
        
        # Substitui ds_motivo_rel pelo ID do motivo (cd_motivo)
        F.col("m.cd_motivo").alias("ds_motivo")
    )
)

display(chamado_limpo)

In [0]:
tb_canal = spark.table("v_credit.silver.tb_canal")

chamado_limpo = (
    chamado_limpo
    .withColumn("cd_canal", F.upper(F.trim(F.col("cd_canal"))))
    
    .withColumn("cd_canal", F.regexp_replace(F.col("cd_canal"), r"(?i)U\.?R\.?A\.?", "URA"))
    
    .withColumn("cd_canal", F.regexp_replace(F.col("cd_canal"), r"(?i)CHAT\s?BOT", "CHATBOT"))

    .withColumn("cd_canal", F.regexp_replace(F.col("cd_canal"), r"(?i)^ATEND.*INICIAL$", "ATENDIMENTO INICIAL"))
    
    .withColumn("cd_canal", F.regexp_replace(F.col("cd_canal"), r"(?i)^ATEND.*ESPECIALIZADO$", "ATENDIMENTO ESPECIALIZADO"))
)

chamado_validacao = (
    chamado_limpo.alias("c")
    .join(
        tb_canal.alias("can"),
        F.col("c.cd_canal") == F.col("can.nm_canal"),
        "left"
    )
    .select(
        *[F.col(f"c.{col}") for col in chamado_limpo.columns if col != "cd_canal"],
        
        F.col("can.cd_canal").alias("cd_canal")
    )
)

display(chamado_validacao)